In [ ]:
# CART on my dataset
from random import seed
from random import randrange
import csv
from csv import reader


from itertools import cycle, islice

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import signal
from sklearn.model_selection import train_test_split

from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
import pandas
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
import numpy as np
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, auc
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from pandas import read_csv
import numpy as np
from numpy import set_printoptions
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

In [ ]:

def best_split_point(X, y, column):
    # sorting y by values of X makes
    # it is almost trivial to count classes
    # above or below candidate split point
    
    # sort X array
    ordering = np.argsort(X[:, column])
    # classes is y in terms of sorted X array
    classes = y[ordering]

    # these vectors tell how many of each
    # class are present below (to the left)
    # of any given candidate splt point
    class_0_below = (classes == 0).cumsum()
    class_1_below = (classes == 1).cumsum()
    # Subtracting the cummulative sum from the total
    # gives us the reversed cummulative sum. These
    # are how many of each class are above (to the
    # right) of any given candidate split point.
    #
    # Because class_0_below is a cummulative sum
    # the last value in the array is the total sum.
    # That means we don't need to make another pass
    # through the array just to get the total; we can
    # just grab the last element. 
    class_0_above = class_0_below[-1] - class_0_below
    class_1_above = class_1_below[-1] - class_1_below
    
    # below_total = class_0_below + class_1_below
    below_total = np.arange(1, len(y)+1)
    # above_total = class_0_above + class_1_above
    above_total = np.arange(len(y)-1, -1, -1)
    # we can now calculate Gini impurity in a single
    # vectorized operation. The naive formula would be:
    #
    #     (class_1_below/below_total)*(class_0_below/below_total)
    # 
    # however, divisions are expensive and we can get this down
    # to only one division if we combine the denominator term.
     # class left/ total_left + class rigth/total_right
    gini = class_1_below * class_0_below / (below_total ** 2) + class_1_above * class_0_above / (above_total ** 2)
    
    # gini index is nan is 1
    gini[np.isnan(gini)] = 1
    # we need to reverse the above sorting to
    # get the rule into the form C_n < split_value. 
    best_split_rank = np.argmin(gini)  # minimum gini (0) is the best split rank, correct classification
    best_split_gini = gini[best_split_rank] 
    best_split_index = np.argwhere(ordering == best_split_rank).item(0)
    best_split_value = X[best_split_index, column] 
    
    return best_split_gini, best_split_value, column


In [ ]:
class Node:
    
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.is_leaf = True
        self.column = None
        self.split_point = None
        self.children = None
         
    def formatted(self, indent=0):
        if self.is_leaf:
            s = "Leaf({p[0]:.3f}, {p[1]:.3f})".format(p=self.probabilities())
        else:
            s = "Branch(X{column} <= {split_point})\n{left}\n{right}".format(
                column=self.column, 
                split_point=self.split_point,
                left=self.children[0].formatted(indent+1),
                right=self.children[1].formatted(indent+1))
        
        return "    " * indent + s

    def __str__(self):
        return self.formatted()

    def __repr__(self):
        return str(self)
    
    def is_pure(self):
        p = self.probabilities()
        if p[0] == 1 or p[1] == 1:  # if probability is 1, then it is pure
            return True
        return False

    def split(self, depth=0):
        X, y = self.X, self.y 
        # if its a leaf node and not pure == 1
        if self.is_leaf and not self.is_pure():
            splits = [ best_split_point(X, y, column) for column in range(X.shape[1])]  # best split from X
            # order splits
            splits.sort()
            #  best gini, split point and column is from 0 element
            gini, split_point, column = splits[0]
            # set splits leaf to false
            self.is_leaf = False
            self.column = column
            self.split_point = split_point # left, right split point
            below = X[:,column] <= split_point
            above = X[:,column] > split_point
            # root Node
            self.children = [
                Node(X[below], y[below]),
                Node(X[above], y[above])
            ]
            # if depth exist, split the child node
            if depth:
                for child in self.children:
                    child.split(depth-1)
    
    def probabilities(self):
    # return array with index of mean of class 0,1
        return np.array([
            np.mean(self.y == 0),
            np.mean(self.y == 1),
        ])

    def predict_proba(self, row):
    # if it is a leaf node retun proabalities 
        if self.is_leaf:
            return self.probabilities()
        else:
        # if row value is less than split point, return prob of 0/left root node, otherwise its the right
            if row[self.column] <= self.split_point:
                return self.children[0].predict_proba(row)
            else:
                return self.children[1].predict_proba(row)



In [ ]:
class Decision_TreeClassifier:
    # initialize max_depth
    def __init__(self, max_depth = 3):
        # initialize max depth as integer
        self.max_depth = int(max_depth)
        # no root node
        self.root = None
        
    # fit the model
    def fit(self, X, y):
        # define root Node
        self.root = Node(X, y)
        self.root.split(self.max_depth)
    
    def predict_proba(self, X):
        results = [] # initialize empty array
        for row in X:
            p = self.root.predict_proba(row)  # return predict value 1 using root node
            # add p list to results
            results += [p]
        # return the list of results
        return np.array(results)
    
    def predict(self, X):
        # probabilities above 0.5 are int type
        return (self.predict_proba(X)[:, 1] > 0.5).astype(int)
    

In [ ]:
filename = "/content/stress_analysis.csv"
names = ['Mean HR','SDNN','RMSSD','NN50','PNN50','SD1','SD2','ApEn',
         'VLF(ms^2)','LF(ms^2)','HF(ms^2)','LF/HF ratio','TP',
         'VLF_lomb(ms^2)','LF_lomb(ms^2)','HF_lomb(ms^2)','LF/HF_lomb ratio','TP_lomb','Stress']
dataframe = read_csv(filename, names =names)
df = dataframe
df.head(5)
# df.drop(['Mean HR','SDNN','RMSSD','PNN50','SD1','SD2','ApEn','VLF(ms^2)','LF(ms^2)','HF(ms^2)','TP','LF_lomb(ms^2)','VLF_lomb(ms^2)','LF/HF_lomb ratio','TP_lomb'], axis = 1, inplace = True)
array = dataframe.values
X = np.array(dataframe.iloc[:,0:18])
y = np.array(dataframe.iloc[:,18])
y=y.reshape(-1,1)
# scaler = MinMaxScaler(feature_range = (0,1)) # scale the values to min 0, max 1
# rescaledX = scaler.fit_transform(X) # fit the trainning feature X into scaler
# # #summarize the transformed data

# scaler = MinMaxScaler(feature_range = (0,1)) # scale the values to min 0, max 1
# rescaled_y = scaler.fit_transform(y)
# set_printoptions(precision=3) # how many decimal places.
# print(rescaledX[0:5,:])


In [ ]:
dt = DecisionTreeClassifier(max_depth = 2)
dt.fit(X,y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=2, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
model = Decision_TreeClassifier(max_depth=2)
model.fit(X, y)
y_hat = model.predict(X)
p_hat = model.predict_proba(X)[:,1]

print(confusion_matrix(y, y_hat))
print('Acc:', accuracy_score(y, y_hat))

[[28  0]
 [ 6 15]]
Acc: 0.8775510204081632


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
def classify(model, Iterations, num_folds, rescaledX, y):
    acc = []
    pre = []
    rec = []

    for i in range(Iterations):

        # Shuffle the dataset for each iteration
        data = list(zip(rescaledX,y))
        random.shuffle(data)
        rescaledX, y = zip(*data)
        rescaledX = np.array(rescaledX)
        y = np.array(y)
        #y = y.reshape(-1,1)

        # Perform 5-fold validation
        kfold = KFold(n_splits = num_folds)#, shuffle=True, random_state = None)
        #results = cross_val_score(DT, rescaledX, y, cv = kfold)
        #print(results)

        for train_index, test_index in kfold.split(rescaledX):
            X_train, X_test = rescaledX[train_index], rescaledX[test_index]
            y_train, y_test = y[train_index], y[test_index]

            # perform training and testing
            if model == SVC:
                model.fit(X_train,y_train)
            else:
                model.fit(X_train,y_train.ravel())
            #dtscores = DT.score(X_test,y_test)
            yPred = model.predict(X_test)


            # record performance
            acc = np.append(acc,metrics.accuracy_score(y_test, yPred)) 
            pre = np.append(pre,metrics.precision_score(y_test, yPred, pos_label=1, average='macro'))
            rec = np.append(rec,metrics.recall_score(y_test, yPred, pos_label=1, average='macro'))

    return acc, pre, rec
    


In [ ]:
import pandas as pd
import numpy as np
from numpy import set_printoptions

# Preparing dataset
from sklearn.preprocessing import MinMaxScaler
import random

# Models
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Cross Validation
from sklearn.model_selection import KFold

# Performance Measure
#from sklearn.model_selection import cross_val_score
#from sklearn.metrics import confusion_matrix
#from sklearn.utils.multiclass import unique_labels
from sklearn import metrics
import statistics

# Plotting
import matplotlib.pyplot as plt
# Gradient Boost
GB = GradientBoostingClassifier(n_estimators=100, 
                                learning_rate=1.0, 
                                max_depth=1, 
                                random_state=0)

# Support Vector Machine
SVM = SVC(random_state = 0, gamma='scale',probability = True)

# Linear Discrimant Analysis
LDA = LinearDiscriminantAnalysis()

NB = GaussianNB()

In [ ]:
acc, pre, rec = classify(GB, 1, 10,  X, y)
print('Accuracy (mean, std):', statistics.mean(acc), statistics.stdev(acc))
print('Precision (mean, std):', statistics.mean(pre), statistics.stdev(pre))
print('Recall (mean, std):', statistics.mean(rec), statistics.stdev(pre))

Accuracy (mean, std): 0.8150000000000001 0.17646214576755234
Precision (mean, std): 0.8333333333333333 0.19543398999264291
Recall (mean, std): 0.8041666666666667 0.19543398999264291


In [ ]:
NB.fit(X, y)
y_hat = NB.predict(X)
p_hat = NB.predict_proba(X)[:,1]

print(classification_report(y, y_hat))

              precision    recall  f1-score   support

         0.0       0.60      0.93      0.73        28
         1.0       0.67      0.19      0.30        21

    accuracy                           0.61        49
   macro avg       0.64      0.56      0.51        49
weighted avg       0.63      0.61      0.55        49



/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
acc, pre, rec = classify(SVM, 1, 1,  X, y)
print('Accuracy (mean, std):', statistics.mean(acc), statistics.stdev(acc))
print('Precision (mean, std):', statistics.mean(pre), statistics.stdev(pre))
print('Recall (mean, std):', statistics.mean(rec), statistics.stdev(pre))

Accuracy (mean, std): 0.555 0.2753280871170894
Precision (mean, std): 0.33 0.25975415727611023
Recall (mean, std): 0.525 0.25975415727611023


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Pr

In [ ]:
acc, pre, rec = classify(NB, 1, 1,  X, y)
print('Accuracy (mean, std):', statistics.mean(acc), statistics.stdev(acc))
print('Precision (mean, std):', statistics.mean(pre), statistics.stdev(pre))
print('Recall (mean, std):', statistics.mean(rec), statistics.stdev(pre))

Accuracy (mean, std): 0.5650000000000001 0.2728959915832811
Precision (mean, std): 0.4325 0.31070037370788955
Recall (mean, std): 0.5175 0.31070037370788955


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision 

In [ ]:
# personalized model

acc, pre, rec = classify(model, 1, 1,  X, y)
print('Accuracy (mean, std):', statistics.mean(acc), statistics.stdev(acc))
print('Precision (mean, std):', statistics.mean(pre), statistics.stdev(pre))
print('Recall (mean, std):', statistics.mean(rec), statistics.stdev(pre))

Accuracy (mean, std): 0.65 0.14337208778404378
Precision (mean, std): 0.6833333333333333 0.16220890356284925
Recall (mean, std): 0.6833333333333333 0.16220890356284925


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in true_divide
